In [43]:
from shapely.geometry import Polygon
from shapely import wkt
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

In [44]:
# amewoo, lobcur, buwtea, tunswa, sursco, bkpwar, sheowl, snoowl1
species = 'lobcur'
model = 'favorite chain'
model1 = 'chain'

resolution = 4

In [45]:
# Choose a departure cell (red)
departure_cell = "844873bffffffff"

# choose a destination cell (blue)
destination_cell = "84299bbffffffff"

In [46]:
df = pd.read_csv(f'{species}/tables/h3_abundance.csv')
df['geometry'] = df['geometry'].apply(wkt.loads)
df = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

In [47]:
flow = pd.read_csv(f'{species}/tables/{model}/{model}_0.1.csv')
flow1 = pd.read_csv(f'{species}/tables/{model1}/{model1}_0.1.csv')

In [48]:
# Filter flow data for that departure cell
flow_filtered = flow[flow["departure_cell"] == departure_cell].copy()

# Compute probability for each destination cell
flow_filtered["probability"] = flow_filtered["flow"] / flow_filtered["departure_value"]

# Merge probabilities into df using destination cell
df_plot = df.copy()
df_plot = df_plot.merge(
    flow_filtered[["destination_cell", "probability"]],
    left_on="cell",
    right_on="destination_cell",
    how="left"
)

df_plot['probability'] = df_plot['probability'].fillna(0)

# Assign edge color: red for departure cell, grayscale for others based on probability
def edge_color(row):
    if row["cell"] == departure_cell:
        return "red"
    elif row["cell"] == destination_cell:
        return "blue"
    else:
        return (0, 0, 0, 0)  # fully transparent

df_plot["edgecolor"] = df_plot.apply(edge_color, axis=1)

# Plot
fig, ax = plt.subplots(figsize=(30, 10))
df_plot.plot(
    column="probability",
    linewidth=0.5,
    edgecolor=df_plot["edgecolor"],
    ax=ax,
    legend=True
)
plt.savefig(f"{species}/plots/cell_probabilities.pdf", dpi=300, bbox_inches="tight")
plt.close()

In [49]:
# Filter flow data for that departure cell
flow1_filtered = flow1[flow1["departure_cell"] == departure_cell].copy()

# Compute probability for each destination cell
flow1_filtered["probability"] = flow1_filtered["flow"] / flow1_filtered["departure_value"]

# Merge probabilities into df using destination cell
df_plot = df.copy()
df_plot = df_plot.merge(
    flow1_filtered[["destination_cell", "probability"]],
    left_on="cell",
    right_on="destination_cell",
    how="left"
)

df_plot['probability'] = df_plot['probability'].fillna(0)

# Assign edge color: red for departure cell, grayscale for others based on probability
def edge_color(row):
    if row["cell"] == departure_cell:
        return "red"
    elif row["cell"] == destination_cell:
        return "blue"
    else:
        return (0, 0, 0, 0)  # fully transparent

df_plot["edgecolor"] = df_plot.apply(edge_color, axis=1)

# Plot
fig, ax = plt.subplots(figsize=(30, 10))
df_plot.plot(
    column="probability",
    linewidth=0.5,
    edgecolor=df_plot["edgecolor"],
    ax=ax,
    legend=True
)
plt.savefig(f"{species}/plots/cell_probabilities1.pdf", dpi=300, bbox_inches="tight")
plt.close()